# ANN(인공신경망)

In [1]:
from keras import layers, models
# layers는 계층을 만드는 모듈
# models는 layer들의 연결, 신경망모델생성, 컴파일, 학습
# models.Model객체에서 딥러닝 처리함수를 제공


Using TensorFlow backend.


## 데이터 불러오기


데이터 활용도에 따라 3가지

### 학습, 검증, 평가 데이터
학습 : 모델학습
검증 : 학습시 성능 검증(학습데이터안에서 일정비율로 추출, 혹은 바로 평가데이터 직접 사용)
평가 : 학습후 모델 성능 평가



In [2]:
import numpy as np  #reshape()
from keras import datasets  # mnist()
from keras.utils import np_utils  # to_categorical()


def Data_func():
    (X_train, y_train), (X_test, y_test) = datasets.mnist.load_data()

    Y_train = np_utils.to_categorical(y_train) #0과1로 표현되는 백테 10개로 바꿈
    Y_test = np_utils.to_categorical(y_test)   #0과1로 표현되는 백테 10개로 바꿈
    #분류작업 시 정수보다 이진 벡터로 출력 변수구성이 효율적
    

    #아래 3줄은 실습 및 평가용 데이터를 3차원에서 2차원으로 조정
    L, W, H = X_train.shape #멤버변수 shape에는 2D 이미지데이터를 저장하는 저장소 규격이 들이있다.
    X_train = X_train.reshape(-1, W * H) #벡터 이미지 혀태로 바꾸어야 한다.
    X_test = X_test.reshape(-1, W * H) #-1:행렬의행을자동설정, 전체 운소수에서 W*H 자동으로 나눠줌

    X_train = X_train / 255.0 #ANN의 최적화를 위해 아규먼트를 정규화함
    X_test = X_test / 255.0 #결국 0 ~ 1사이의 실수로 바꾼다.

    return (X_train, Y_train), (X_test, Y_test)


## ANN학습 결과 그래프 구현
분석한 결과를 바탕으로 바른 학습인지
하이퍼파라미터를 어떻게 조절하는지
하이퍼파라미터는 학습 전에 사람이 지정해주어야 하는 변수
측정한 손실과
정확도의 추이를 관찰


## ANN구조와상관없이 다른 신경마에서도 이함수들을 사용가능

In [3]:
import matplotlib.pyplot as plt


def plot_acc(history, title=None):  #정확도를 그리는 함수
    # summarize history for accuracy
    if not isinstance(history, dict):
        history = history.history

    plt.plot(history['accuracy'])
    plt.plot(history['val_accuracy'])
    if title is not None:
        plt.title(title)
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Training', 'Verification'], loc=0)
    # plt.show()


def plot_loss(history, title=None): #손실을 그리는 함수
    # summarize history for loss
    if not isinstance(history, dict):
        history = history.history

    plt.plot(history['loss']) # 선 그리기 #history에 들어있는 실제 학습 데이터로 구한 손실값
    plt.plot(history['val_loss'])       #학습 데이터 일부를 사용한 검증 데이터로 구한 손실값
    if title is not None:
        plt.title(title) #그래프 제목 표시
    plt.ylabel('Loss')  #X축이름표시
    plt.xlabel('Epoch') #Y축이름표시
    plt.legend(['Training', 'Verification'], loc=0) #각 라인의 표식 표시
    # plt.show()



In [5]:
class ANN_models_class(models.Model): #model.Model로 부터 특성 상속
                                    #model.Model 학습, 예측, 평가와 같은 다양한 함수 제공
    def __init__(self, Nin, Nh, Nout): #초기화함수 정의
                    # 입력,  은닉, 출력계층의 노드 수를 받는다.
            
            
        # Prepare network layers and activate functions
        hidden = layers.Dense(Nh) #은닉계층이 하나이므로 은닉의 출려도 한개
        #hidden_l = [layers.Dense(n) for n in Nh_l] # 단일문장
        #hidden_l = []  # 이하 여러문장
        #for n in Nh_l
            #hidden_l.append(layers.Dense(n))
            
        output = layers.Dense(Nout) #노드 수가 Nout개인 출력 계층을 정의
                                    #모델을 구성하는 요소로 사용됨
            
        relu = layers.Activation('relu') #비선형을 넣어준다.
        softmax = layers.Activation('softmax') #분류의 경우 사용
        #위 함수 2개는 계산 모듈이므로 한 번 정의해두면 모델내에서 여러번 사용할수 있따.
        
        # Connect network elements
        x = layers.Input(shape=(Nin,))
        h = relu(hidden(x))
        y = softmax(output(h))

        super().__init__(x, y) #상속받은 부모 클래스의 초기화 진행
        self.compile(loss='categorical_crossentropy',
                     optimizer='adam', metrics=['accuracy'])


In [6]:
def main():
    Nin = 784 #입력 길이가 784인 데이터
    Nh = 100 #은닉계층의 노드수를 100d으로
    number_of_class = 10 # 분류할 데이터 클래스 수와 같다.
    Nout = number_of_class

    #model = ANN_models_func(Nin, Nh, Nout)
    model = ANN_models_class(Nin, Nh, Nout)
    # model = ANN_seq_class(Nin, Nh, Nout)  #모델인스턴스 만들고
    (X_train, Y_train), (X_test, Y_test) = Data_func() #데이터 불러오기

    ##############################################
    # Training
    ##############################################
    history = model.fit(X_train, Y_train, epochs=5, batch_size=100, validation_split=0.2)
    #학습 진행상황을 변수 history에 저장
    # batch_size는 한데이터를얼마씩나눠서 넣을지를 지정
    # validation_split 전체학습데이터 중에서학습진행중 성능검증에 데이터를얼마나사용할지를결정하는변수입니다. 
    # 학습데이터의20%를성능검증에활용
    
    performace_test = model.evaluate(X_test, Y_test, batch_size=100)
    print('Test Loss and Accuracy ->', performace_test)

    #plot_loss(history)
    #plt.show()
    #plot_acc(history)
    #plt.show()


# Run code
if __name__ == '__main__':
    main()


Train on 48000 samples, validate on 12000 samples
Epoch 1/5
48000/48000 [==============================] - 1s 15us/step - loss: 0.4047 - accuracy: 0.8889 - val_loss: 0.2172 - val_accuracy: 0.9417
Epoch 2/5
48000/48000 [==============================] - 1s 13us/step - loss: 0.1939 - accuracy: 0.9445 - val_loss: 0.1627 - val_accuracy: 0.9536
Epoch 3/5
48000/48000 [==============================] - 1s 13us/step - loss: 0.1431 - accuracy: 0.9593 - val_loss: 0.1335 - val_accuracy: 0.9617
Epoch 4/5
48000/48000 [==============================] - 1s 13us/step - loss: 0.1113 - accuracy: 0.9681 - val_loss: 0.1178 - val_accuracy: 0.9650
Epoch 5/5
10000/10000 [==============================] - 0s 5us/step
Test Loss and Accuracy -> [0.10729601367842406, 0.968500018119812]
